In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
from itertools import chain
import glob

In [3]:
df_merged = pd.read_excel('list_all_em_GTO.xlsx')

In [4]:
df_merged.head()

,Unnamed: 0,Register Date,HS Code,Product Description,Producer,Brand,Place of Receipt,Country of Origin,Country of Sales,Shipper,Shipper Address,Consignee,Consignee Address,Quantity,Unit of Measure,Incoterms,Gross Weight in KG,Net Weight in KG,Cargo Value USD
0,0,2011-01-29,8407349908,СИЛОВОЙ АГРЕГАТ TOYOTA 2TR-FE-261 В РАЗОБРАННО...,TOYOTA TSUSHO CORPORATION,ОТСУТСТВУЕТ,КОТКА,"JP, JPN, Japan","JP, JPN, Japan",TOYOTA TSUSHO CORPORATION,"450-8575 NAKAMURA-KU NAGOYA 9-8,MEIEKI 4-CHOME",ООО ТЕХНОСЕРВИСПРОМ,"603104, НИЖЕГОРОДСКАЯ ОБЛ, Н.НОВГОРОД, НАРТОВА...",15,ШТ,CIF,3604.0,3017.0,73063.36
1,1,2011-01-29,8407349908,СИЛОВОЙ АГРЕГАТ TOYOTA 2TR-FE-A343E-111 В РАЗО...,TOYOTA TSUSHO CORPORATION,ОТСУТСТВУЕТ,КОТКА,"JP, JPN, Japan","JP, JPN, Japan",TOYOTA TSUSHO CORPORATION,"450-8575 NAKAMURA-KU NAGOYA 9-8,MEIEKI 4-CHOME",ООО ТЕХНОСЕРВИСПРОМ,"603104, НИЖЕГОРОДСКАЯ ОБЛ, Н.НОВГОРОД, НАРТОВА...",5,ШТ,CIF,1765.0,1465.0,37014.64
2,2,2011-01-17,8407343009,"ДВИГАТЕЛЬ ЗИЛ ,Б/У,БЕНЗИНОВЫЙ,ОБЪЕМ 6000СМ.КУБ...",АМО ЗИЛ,ЗИЛ,ЕКАТЕРИНБУРГ,"RU, RUS, Russia","UA, UKR, Ukraine",ООО ПЕТРУС,"Г.ДРОГОБЫЧ УЛ.ГАЙДАМАЦКАЯ, 7/1",ЗАО АВТОМОБИЛИ И МОТОРЫ УРАЛА,"624130, СВЕРДЛОВСКАЯ ОБЛ., Г.НОВОУРАЛЬСК, УЛ.А...",30,ШТ,CPT,14400.0,14400.0,27000.00
3,3,2011-01-17,8407343009,"ДВИГАТЕЛЬ ЗИЛ ,Б/У,НЕКОМПЛЕКТНЫЙ,БЕНЗИНОВЫЙ,ОБ...",АМО ЗИЛ,ЗИЛ,ЕКАТЕРИНБУРГ,"RU, RUS, Russia","UA, UKR, Ukraine",ООО ПЕТРУС,"Г.ДРОГОБЫЧ УЛ.ГАЙДАМАЦКАЯ, 7/1",ЗАО АВТОМОБИЛИ И МОТОРЫ УРАЛА,"624130, СВЕРДЛОВСКАЯ ОБЛ., Г.НОВОУРАЛЬСК, УЛ.А...",15,ШТ,CPT,3000.0,3000.0,6894.50
4,4,2011-01-20,8407343009,ДВИГАТЕЛИ БЕНЗИНОВЫЕ В СБОРЕ С КОРОБКОЙ ПЕРЕДА...,MERCEDES - BENZ,MERCEDES - BENZ,ТОЯМА НЬЮ ПОРТ,"JP, JPN, Japan","JP, JPN, Japan","METAL AUTO HOKURIKU CO., LTD","ISHIKAWA KEN KOMATSU SHI SHIRAE MACHI, RO 62",ИП КОЧЕТКОВ ВЯЧЕСЛАВ ВИКТОРОВИЧ,"664019, , Г ИРКУТСК, ПЕР. МИНУСИНСКИЙ Д 10",38,ШТ,FOB,6460.0,6460.0,6536.45


In [5]:
df_merged.drop(columns=['Unnamed: 0'],inplace=True)

In [6]:
for col in list(df_merged.columns):
 (df_merged[col].dtype ==  'object') and (df_merged[col].fillna(value='unknown',inplace = True)) or (df_merged[col].fillna(value=0, inplace = True))

In [8]:
df_merged.isnull().sum()

Register Date          0
HS Code                0
Product Description    0
Producer               0
Brand                  0
Place of Receipt       0
Country of Origin      0
Country of Sales       0
Shipper                0
Shipper Address        0
Consignee              0
Consignee Address      0
Quantity               0
Unit of Measure        0
Incoterms              0
Gross Weight in KG     0
Net Weight in KG       0
Cargo Value USD        0
dtype: int64

In [9]:
def extract_engine_model (description, list_em):
    engine_model = []
    for e in list_em:
        em_reg1 = re.findall(r"(\b{}-\w+\b)".format(e), description)
        em_reg2 = re.findall(r"(\b{}-\w+-\w+\b)".format(e), description)
        if len(em_reg1) != 0 :
          em_reg1 = list(set(em_reg1)) 
          engine_model.append(em_reg1)
        if len(em_reg2) != 0 :
          em_reg2 = list(set(em_reg2))
          engine_model.append(em_reg2)
    
    return list(chain(*engine_model))

In [20]:
list_all_engine_model = []

In [38]:
list_toyota = pd.read_excel('em_toyota.xlsx')

In [46]:
list_all_engine_model_toyota = []

for des in df_merged['Product Description'].tolist():
    list_em = extract_engine_model(des,list_toyota['List engine'].str.strip().tolist())
    if len(list_em) != 0:
      list_all_engine_model_toyota.append(list_em)
    
list_all_engine_model_toyota

[['2TR-FE', '2TR-FE-261'],
 ['2TR-FE', '2TR-FE-A343E'],
 ['1JZ-0649357'],
 ['5K-1298931', '2TR-0221092', '2UZ-9064542'],
 ['5K-1298931', '2TR-0221092', '2UZ-9064542'],
 ['5VZ-0970651', '5VZ-0799452'],
 ['1FZ-0331710', '5VZ-1157742', '5VZ-0818405'],
 ['3S-7968456'],
 ['1G-5897371'],
 ['3F-0309291'],
 ['1JZ-6119500', '1JZ-6116172', 'V-2491'],
 ['5S-0922401', '1ZZ-1052247', '1ZZ-0972897', '1G-6613201'],
 ['1NZ-B366087', '1NZ-A946272'],
 ['3S-7051240', '2TZ-1423316', '1JZ-1423316'],
 ['3S-1303230', '3S-6911045', '3S-7306959', '4S-1424182', '1NZ-2618515'],
 ['2JZ-0299325'],
 ['1MZ-0819394', '2UZ-0024244', '2UZ-0012648', '2UZ-0076639', '2UZ-0620456'],
 ['5A-G855525', '3S-7526819', '4S-1329002', '4E-2277208', '5E-1403456'],
 ['3S-9278941'],
 ['5VZ-1213852', '2UZ-0005297', '2UZ-9003553'],
 ['1ZZ-0399079', '2NZ-2404695', '1MZ-0762158'],
 ['3S-9192911'],
 ['1NZ-B000831', '1JZ-0523462', '1MZ-0792353'],
 ['1NZ-A372882', '1AZ-4016158'],
 ['2SZ-1465429', '2JZ-0965689'],
 ['3S-6669363', '1G-6365381']

In [47]:
len(list_all_engine_model_toyota)

25699

In [48]:
all_engine_model_toyota = list(chain(*list_all_engine_model_toyota))
len(all_engine_model_toyota)

93048

In [42]:
df_list_engine_toyota = df_merged['Product Description'].map(lambda x : extract_engine_model(x,list_toyota['List engine'].str.strip().tolist()))

In [43]:
df_list_engine_toyota

0           [2TR-FE, 2TR-FE-261]
1         [2TR-FE, 2TR-FE-A343E]
2                             []
3                             []
4                             []
                   ...          
259191                        []
259192                        []
259193                        []
259194                        []
259195                        []
Name: Product Description, Length: 259196, dtype: object

In [16]:
list_honda = pd.read_excel('em_honda.xlsx')

In [17]:
list_honda.drop_duplicates()

,Engine_Famlily
0,E07A
1,ECA1
2,E05A
4,E07Z
5,S07A
...,...
276,RA271E
277,RA272E
278,RA273E
281,RA121E


In [19]:
list_all_engine_model_honda = []

for des in df_merged['Product Description'].tolist():
    list_em = extract_engine_model(des,list_honda['Engine_Famlily'].str.strip().tolist())
    list_all_engine_model_honda.append(list_em)
    
list_all_engine_model_honda

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['J30A-6000582'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['K20A-2080249'],
 [],
 [],
 [],
 [],
 ['B20B-1161655'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['B20B-1023753', 'L13A-1001826'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L13A-1602007'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['J30A-3004572'],
 ['K24A-2508078'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 

In [21]:
list_mazda = pd.read_excel('em_mazda.xlsx')

In [22]:
list_all_engine_model_mazda = []

for des in df_merged['Product Description'].tolist():
    list_em = extract_engine_model(des,list_mazda['List engine'].str.strip().tolist())
    list_all_engine_model_mazda.append(list_em)
    
list_all_engine_model_mazda

[['FE-261'],
 ['FE-A343E', 'FE-A343E-111'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L3-203494'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['FE-943620', 'KL-701503'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L3-239057'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['G5-206157'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['FP-776614'],
 ['FP-701420', 'ZL-306871'],
 [],
 [],
 [],
 [],
 [],
 ['ZJ-261764'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['FP-766535'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [23]:
len(list_all_engine_model_mazda)

259196

In [24]:
df_list_engine = pd.read_excel('C:\\Users\\Admin\\Github\\Kaiho\\Engine Model _1025.xlsx')
df_list_engine.head()

,Unnamed: 0,メーカー,燃料,買付受付,型式,駆動1,駆動2,駆動3,詳細,車輌型式,KRA売単価,車名,仕入値,EG売価\n（マニ触付）\nW,EG売価\n（マニ触抜/無）\nWO
0,2.0,FIAT,0,×,4,FR,AT,2WD,NaN,NaN,NaN,NaN,NaN,売価確認,売価確認
1,3.0,PORCHE,NaN,×,64,RR,AT,4WD,NaN,NaN,NaN,NaN,NaN,売価確認,0
2,4.0,PORCHE,NaN,×,66,RR,MT,2WD,NaN,NaN,NaN,NaN,NaN,売価確認,0
3,5.0,PORCHE,NaN,×,70,RR,AT,4WD,NaN,NaN,NaN,NaN,NaN,売価確認,0
4,6.0,BENZ,NaN,○,102,FR,AT,2WD,NaN,NaN,NaN,NaN,NaN,40000,0


In [25]:
df_count_brand_toyota =  df_list_engine[df_list_engine['メーカー']  == 'TOYOTA']

In [26]:
list_toyota = list(df_count_brand_toyota['型式'].unique())

In [28]:
list_all_engine_model_toyota_kaiho = []

for des in df_merged['Product Description'].tolist():
    list_em = extract_engine_model(des,list_toyota)
    if len(list_em) !=0:
      list_all_engine_model_toyota_kaiho.append(list_em)
    
list_all_engine_model_toyota_kaiho

[['2TR-FE-261'],
 ['2TR-FE-A343E', '2TR-FE-A343E-111'],
 ['5K-1298931'],
 ['5K-1298931'],
 ['3F-0309291'],
 ['3Y-1087320'],
 ['1ZZ-FE-1075334'],
 ['1N-0164680'],
 ['2L-4744731'],
 ['7K-0535452'],
 ['3Y-1131701'],
 ['7K-0305900'],
 ['3Y-0960114'],
 ['5A-FE-4479404'],
 ['5A-FE-4479404'],
 ['5A-FE-4479404'],
 ['7K-0384461'],
 ['3Y-1087320'],
 ['1RZ-2972261'],
 ['3Y-2025152'],
 ['13B-472259'],
 ['T2-007921'],
 ['B-3130483'],
 ['7K-0441751'],
 ['3Y-1104798', '3Y-0852568', '7K-0'],
 ['3F-0342176'],
 ['7K-НОМЕР'],
 ['T2-702740'],
 ['1RZ-3074053'],
 ['7K-074', '7K-0317006'],
 ['7K-0388811'],
 ['2Y-0878492'],
 ['3Y-0982805'],
 ['3Y-0809881'],
 ['2Y-0733933'],
 ['3F-0330119'],
 ['3Y-2006752'],
 ['3Y-0908831'],
 ['3Y-1188703'],
 ['T2-008564'],
 ['3Y-1092534'],
 ['7K-0699656'],
 ['2Y-0877867'],
 ['3Y-1048986'],
 ['3Y-1193297'],
 ['7K-0262939'],
 ['T2-702411'],
 ['7K-0217074'],
 ['7K-0484243'],
 ['3Y-1255955'],
 ['1RZ-3053319'],
 ['7K-0530481'],
 ['T2-702775'],
 ['3F-0296216'],
 ['2Y-0858389'],
 ['

In [29]:
len(list_all_engine_model_toyota_kaiho)

966

In [35]:
df_merged.columns

Index(['Register Date', 'HS Code', 'Product Description', 'Producer', 'Brand',
       'Place of Receipt', 'Country of Origin', 'Country of Sales', 'Shipper ',
       'Shipper Address', 'Consignee', 'Consignee Address', 'Quantity',
       'Unit of Measure', 'Incoterms', 'Gross Weight in KG',
       'Net Weight in KG', 'Cargo Value USD'],
      dtype='object')

In [36]:
df_merged

array(['TOYOTA TSUSHO CORPORATION', 'ООО ПЕТРУС ',
       'METAL AUTO HOKURIKU CO., LTD', ...,
       'UAB AFTALITANA (BY ORDER RIJAS GMBH AND CO. KG)',
       'FESCO INTEGRATED TRANSPORY LLC O/B OF KAIHO INDUSTRY CO. LTD',
       'POPLA CO.LTD'], dtype=object)

In [44]:
all_engine_model_toyota_kaiho  = list(chain(*list_all_engine_model_toyota_kaiho))

In [45]:
len(all_engine_model_toyota_kaiho)

1188

In [52]:
df_merged['shipper_extract']  = df_merged['Shipper '].map(lambda x : 'KAIHO' if 'KAIHO' in x else ('Others' if x  else 'None'))

In [70]:
df_merged['country_extract']  = df_merged['Country of Sales'].map(lambda x : 'Japan' if ('japan' in x.lower() or 'jp' in x.lower() or 'jpn' in x.lower()) else ('Others' if x  else 'None'))

In [71]:
df_kaiho_extract = df_merged[(df_merged['shipper_extract'] == 'KAIHO') & (df_merged['country_extract'] == 'Japan')]

In [72]:
df_kaiho_extract

,Register Date,HS Code,Product Description,Producer,Brand,Place of Receipt,Country of Origin,Country of Sales,Shipper,Shipper Address,Consignee,Consignee Address,Quantity,Unit of Measure,Incoterms,Gross Weight in KG,Net Weight in KG,Cargo Value USD,shipper_extract,country_extract
27,2011-01-12,8407343009,"БЕНЗИНОВЫЕ ДВИГАТЕЛИ ДЛЯ А/М, РАБОЧИМ ОБЪЕМОМ ...","DAIHATSU, HONDA, SUBARU, MAZDA, MITSUBISHI, NI...","DAIHATSU, HONDA, SUBARU, MAZDA, MITSUBISHI, NI...",ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO., LTD",920-0209 ISHIKAWA-KEN KANAZAWA-SHI HIGASHI-KAG...,ВИНАКО ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ,"690078, ПРИМОРСКИЙ КРАЙ, ВЛАДИВОСТОК, ПУШКИНСК...",24,ШТ,CFR,5010.0,5010.0,3263.98,KAIHO,Japan
37,2011-01-12,8407343009,"ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ, БЕНЗИНОВЫЕ ДЛЯ...","TOYOTA, NISSAN","TOYOTA, NISSAN",ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO., LTD",920-0209 ISHIKAWA KANAZAWA-SHI 1-25 HIGASHIKAG...,ХМЕЛЕВ ЕВГЕНИЙ АЛЕКСАНДРОВИЧ ИП,"690018, ПРИМОРСКИЙ КРАЙ, Г. ВЛАДИВОСТОК, УЛ. В...",3,ШТ,CFR,750.0,750.0,1570.00,KAIHO,Japan
47,2011-01-13,8407343009,"ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ, БЕНЗИНОВЫЕ ДЛЯ...",MITSUBISHI,MITSUBISHI,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO., LTD",920-0209 ISHIKAWA KANAZAWA-SHI 1-25 HIGASHIKAG...,ХМЕЛЕВ ЕВГЕНИЙ АЛЕКСАНДРОВИЧ ИП,"690018, ПРИМОРСКИЙ КРАЙ, Г. ВЛАДИВОСТОК, УЛ. В...",1,ШТ,CFR,300.0,300.0,450.00,KAIHO,Japan
68,2011-01-17,8407343009,"ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ БЕНЗИНОВЫЕ, Б/У...",TOYOTA,TOYOTA,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO.,LTD.",920-0209 ISHIKAWA KANAZAWA-SHI HIGASHIKAGATSUM...,ООО АЗИЯ,"690105, ПРИМОРСКИЙ КРАЙ, Г. ВЛАДИВОСТОК, УЛ. Р...",7,ШТ,CFR,955.0,955.0,741.25,KAIHO,Japan
69,2011-01-17,8407343009,"ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ БЕНЗИНОВЫЕ, Б/У...",TOYOTA,TOYOTA,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO.,LTD.",920-0209 ISHIKAWA KANAZAWA-SHI HIGASHIKAGATSUM...,ООО АЗИЯ,"690105, ПРИМОРСКИЙ КРАЙ, Г. ВЛАДИВОСТОК, УЛ. Р...",22,ШТ,CFR,2955.0,2955.0,2293.62,KAIHO,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258986,2019-12-01,8407343009,ДВИГАТЕЛЬ БЕНЗИНОВЫЙ ДЛЯ А/М БЫВШИЙ В УПОТРЕБЛ...,FORD,FORD,TOYAMASHINKO,"JP, JPN, Japan","JP, JPN, Japan",FESCO INTEGRATED TRANSPORY LLC O/B OF KAIHO IN...,"920-0209, ISHIKAWA KEN, KANAZAWA-SHI, 1-25 HIG...",ООО АВТОСТОК,"630088, ГОРОД НОВОСИБИРСК, СЕВЕРНЫЙ ПРОЕЗД, Д ...",0,unknown,FOB,1165.0,1135.0,1714.19,KAIHO,Japan
258987,2019-12-01,8407343009,ДВИГАТЕЛЬ БЕНЗИНОВЫЙ ДЛЯ А/М БЫВШИЙ В УПОТРЕБЛ...,NISSAN,NISSAN,TOYAMASHINKO,"JP, JPN, Japan","JP, JPN, Japan",FESCO INTEGRATED TRANSPORY LLC O/B OF KAIHO IN...,"920-0209, ISHIKAWA KEN, KANAZAWA-SHI, 1-25 HIG...",ООО АВТОСТОК,"630088, ГОРОД НОВОСИБИРСК, СЕВЕРНЫЙ ПРОЕЗД, Д ...",0,unknown,FOB,1571.0,1530.0,2310.22,KAIHO,Japan
258988,2019-12-01,8407343009,ДВИГАТЕЛЬ БЕНЗИНОВЫЙ ДЛЯ А/М БЫВШИЙ В УПОТРЕБЛ...,MITSUBISHI SUBARU SUZUKI,MITSUBISHI SUBARU SUZUKI,TOYAMASHINKO,"JP, JPN, Japan","JP, JPN, Japan",FESCO INTEGRATED TRANSPORY LLC O/B OF KAIHO IN...,"920-0209, ISHIKAWA KEN, KANAZAWA-SHI, 1-25 HIG...",ООО АВТОСТОК,"630088, ГОРОД НОВОСИБИРСК, СЕВЕРНЫЙ ПРОЕЗД, Д ...",0,unknown,FOB,2125.0,2070.0,3133.66,KAIHO,Japan
258989,2019-12-01,8407343009,ДВИГАТЕЛЬ БЕНЗИНОВЫЙ ДЛЯ А/М БЫВШИЙ В УПОТРЕБЛ...,TOYOTA,TOYOTA,TOYAMASHINKO,"JP, JPN, Japan","JP, JPN, Japan",FESCO INTEGRATED TRANSPORY LLC O/B OF KAIHO IN...,"920-0209, ISHIKAWA KEN, KANAZAWA-SHI, 1-25 HIG...",ООО АВТОСТОК,"630088, ГОРОД НОВОСИБИРСК, СЕВЕРНЫЙ ПРОЕЗД, Д ...",0,unknown,FOB,3584.0,3500.0,5287.45,KAIHO,Japan


In [56]:
list_all_engine_model_toyota_kaiho_extract = []

for des in df_kaiho_extract['Product Description'].tolist():
    list_em = extract_engine_model(des,list_toyota)
    if len(list_em) !=0:
      list_all_engine_model_toyota_kaiho_extract.append(list_em)
    
list_all_engine_model_toyota_kaiho_extract

[]

In [60]:
a = df_kaiho_extract['Producer'].apply(lambda x : 'TOYOTA' in x).sum()

In [73]:
list_engine_GTO  = pd.read_csv('C:\\Users\\Admin\\Github\\Kaiho\\GTO_JAPAN_engine.csv')

D:\Software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
list_engine_GTO

,Register Date,HS Code,Product Description,Producer,Brand,Place of Receipt,Country of Origin,Country of Sales,Shipper,Shipper Address,...,Unit of Measure,Incoterms,Gross Weight in KG,Net Weight in KG,Cargo Value USD,nor_jp_country,group_hs_code,year,nor_shipper,merge_brand
0,2018-01-01,8407343009,ПРОЧИЕ ДВИГАТЕЛИ С ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫМ ДВ...,SUZUKI,SUZUKI,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan",EARLYBIRD CO.LTD,"JAPAN, TOYAMA-SHI, IWASE, SUWAMACHI 52",...,NaN,CFR,200.0,200.0,302.00,Japan,840734,2018,Other,SUZUKI
1,2018-01-01,8407349908,ПРОЧИЕ ДВИГАТЕЛИ С ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫМ ДВ...,MAZDA MOTOR CORPORATION,MAZDA,ПОХАНГ,"JP, JPN, Japan","JP, JPN, Japan",PACIFIC LOGISTIC KOREA LTD. O/B OF MAZDA MOTOR...,", SEOUL, SEONGDONG-GU, 902 ACE HIGHEND SEONGSU...",...,NaN,FOB,107979.8,66410.0,539455.71,Japan,840734,2018,Other,MAZDA
2,2018-01-01,8407343009,ПРОЧИЕ ДВИГАТЕЛИ С ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫМ ДВ...,"DAIHATSU,SUBARU,SUZUKI","DAIHATSU,SUBARU,SUZUKI",ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","ASTRADE CO.,LTD",", , OSAKA, 1-4-9, SHIMAMACHI, CHYUO-KU",...,NaN,CFR,800.0,800.0,1200.00,Japan,840734,2018,Other,"DAIHATSU,SUBARU,SUZUKI"
3,2018-01-01,8407343009,ПРОЧИЕ ДВИГАТЕЛИ С ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫМ ДВ...,TOYOTA MOTOR CORP.,TOYOTA,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan",YAMAGIN CORPORATION,", JAPAN, TOKYO, DAIICHI SEIMEI BLDG,8F,2-19-3 ...",...,NaN,CFR,399.0,399.0,602.49,Japan,840734,2018,Other,TOYOTA
4,2018-01-01,8407343009,ПРОЧИЕ ДВИГАТЕЛИ С ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫМ ДВ...,TOYOTA,TOYOTA,ВЛАДИВОСТОК,"JP, JPN, Japan","JP, JPN, Japan","ARAI CO., LTD","671-1132, HYOGO, HIMEJI CITY, KANBEI CHO OHTSU...",...,NaN,CFR,2540.0,2540.0,3835.40,Japan,840734,2018,Other,TOYOTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100539,2016-12-01,8407343009,ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИ...,HONDA MOTOR CO. LTD,HONDA,ВОСТОЧНЫЙ,"JP, JPN, Japan","JP, JPN, Japan",SHIJO TRADING,SHIZUOKA 418-0022 2150-2 KOIZUMI FUJINOMIYA,...,NaN,CFR,1897.8,1897.8,2846.70,Japan,840734,2016,Other,HONDA
100540,2016-12-01,8407343009,ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИ...,HONDA MOTOR CORPORATION,HONDA,ВОСТОЧНЫЙ,"JP, JPN, Japan","JP, JPN, Japan","EAST-JAPAN RECYCLING RESOURCES CO.,LTD 21-1 SH...",NaN,...,NaN,CFR,900.0,900.0,1360.41,Japan,840734,2016,Other,HONDA
100541,2016-12-01,8407343009,ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИ...,SUBARU-FUJI HEAVY INDUSTRIES LTD,SUBARU,ВОСТОЧНЫЙ,"JP, JPN, Japan","JP, JPN, Japan",SCP-TRANS LLC,KOBE 658-0032 5 KOYOCHO HIGASHI-NADA-KU RIC WE...,...,NaN,CFR,720.0,714.0,1071.00,Japan,840734,2016,Other,SUBARU
100542,2016-12-01,8407343009,ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИ...,HONDA MOTOR CO.,HONDA,ВОСТОЧНЫЙ,"JP, JPN, Japan","JP, JPN, Japan","KAIHO SANGYO CO.,LTD","KANAZAWA-SHI ISHIKAWA,920-0209 1-25 HIGASHIKAG...",...,NaN,CFR,900.0,900.0,1350.30,Japan,840734,2016,KAIHO,HONDA


In [79]:
list_all_engine_model_toyota_kaiho_extract = []

for des in list_engine_GTO['Product Description'].tolist():
    list_em = extract_engine_model(des,list_toyota)
    if len(list_em) !=0:
      list_all_engine_model_toyota_kaiho_extract.append(list_em)
    
list_all_engine_model_toyota_kaiho_extract

[]